In [ ]:
# 데이터셋 명목형 / 숫자형 나누어서 계산해보기

In [ ]:
# 요일을 one_hot_encoding으로
    # functional 왼쪽 오른쪽으로 나누어서 진행
    # val_loss < loss 보다 작으므로 과적함 => 그러려면 층을 줄여주고 노드수도 줄여주는게 좋다
    # min_max 컬럼별로 맞추어 주자

    # 점심 mlp 따로 저녁 mlp 따로 해서 시작


# Next
# 음식들도 가중치가 있는 값으로 변경후에 진행해보기
    # 시작

In [ ]:
# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os
import pandas as pd

# 새 섹션

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_ln = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/train_one_hot.csv')

test_ln = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/test_one_hot.csv')

In [ ]:
train_ln.columns

Index(['numbers', 'dayoff', 'work', 'outsidework', 'workfhome', 'lunch_t',
       'dinner_t', 'Month', 'Date', 'bob', 'soup', 'main', 'bobd', 'soupd',
       'maind', 'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화'],
      dtype='object')

In [ ]:
# 전체데이터에 minmax가 아니고 
# 각각 컬럼별로 minmax 해주는게 중요하다

In [ ]:
# df["B"] = ( df["B"] - df["B"].mean() ) / (df["B"].std() )

In [ ]:
def feture_scaling(df, scaling_strategy="min-max", column=None):
    if column == None:
        column = [column_name for column_name in df.columns]
    for column_name in column:
        if scaling_strategy == "min-max":
            df[column_name] = ( df[column_name] - df[column_name].min() ) /\
                            (df[column_name].max() - df[column_name].min()) 
        elif scaling_strategy == "z-score":
            df[column_name] = ( df[column_name] - df[column_name].mean() ) /\
                            (df[column_name].std() )
    return df

In [ ]:
test_lunch = test_ln[[ 'numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bob', 'soup', 'main',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]

In [ ]:
test_dinner = test_ln[[ 'numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bobd', 'soupd', 'maind',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]

In [ ]:
feture_scaling(test_lunch, column=['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bob', 'soup', 'main',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화'])

feture_scaling(test_dinner, column=['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bobd', 'soupd', 'maind',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화'])

In [ ]:
# train_ln = [['day','bob','soup','main','present','lunch_t']]
# X_test = [['day','bob','soup','main','present']]

In [ ]:
train_ln

In [ ]:
X = train_ln[['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bob', 'soup', 'main',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]
y = train_ln['lunch_t']

In [ ]:
feture_scaling(X, column = ['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bob', 'soup', 'main',  'present'])

In [ ]:

from sklearn.model_selection import train_test_split


X_train, X_vali, y_train, y_vali = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 

                                                    shuffle=True, 

                                                    random_state=1004)


# Random_Grid_Search

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[X.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mae", optimizer=optimizer)

    return model


In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
# keras_reg.fit(X_train, y_train, epochs=1000,
#               validation_data=(X_vali, y_vali),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=32,
                  validation_data=(X_vali, y_vali),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/32
21/21 [==============================] - 2s 13ms/step - loss: 896.3797 - val_loss: 863.2642
Epoch 2/32
21/21 [==============================] - 0s 5ms/step - loss: 891.5453 - val_loss: 853.9004
Epoch 3/32
21/21 [==============================] - 0s 5ms/step - loss: 871.9510 - val_loss: 814.0087
Epoch 4/32
21/21 [==============================] - 0s 5ms/step - loss: 786.2869 - val_loss: 638.1672
Epoch 5/32
21/21 [==============================] - 0s 5ms/step - loss: 439.6968 - val_loss: 172.5214
Epoch 6/32
21/21 [==============================] - 0s 5ms/step - loss: 173.4344 - val_loss: 157.3256
Epoch 7/32
21/21 [==============================] - 0s 5ms/step - loss: 155.0200 - val_loss: 160.2966
Epoch 8/32
21/21 [==============================] - 0s 4ms/step - loss: 142.6319 - val_loss: 130.0181
Epoch 9/32
21/21 [==============================] - 0s 4ms/step - loss: 127.3554 - val_loss: 120.0751
Epoch 10/32
21/21 [=

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f0d694bedd0>,
                   param_distributions={'learning_rate': [0.001683454924600351,
                                                          0.02390836445593178,
                                                          0.008731907739399206,
                                                          0.004725396149933917,
                                                          0.0006154014789262348,
                                                          0.0006153331256530192,
                                                          0.0003920021771415983,
                                                          0.01619845322936229,
                                                          0.004779156784872302,
                                                          0.007821074275112298,...
                                                          0.00502142573

In [ ]:
rnd_search_cv.best_params_

{'learning_rate': 0.022174573948353458, 'n_hidden': 1, 'n_neurons': 4}

In [ ]:
# 예측
lunch = rnd_search_cv.predict(test_lunch)

In [ ]:
lunch

array([ 802.75995,  663.1075 ,  406.64917, 1267.902  , 1017.93634,
       1019.8756 ,  922.80396,  694.95544, 1280.4279 , 1004.3019 ,
        869.4614 , 1282.0868 , 1089.5444 , 1055.6483 ,  888.84326,
        670.24066, 1226.7225 ,  985.5719 ,  954.49756,  843.27484,
        583.7357 ,  772.1982 ,  828.3764 ,  695.75586,  480.729  ,
       1063.004  ,  833.623  ,  836.75543,  671.7038 ,  451.73303,
       1079.7985 ,  860.2123 ,  820.7321 ,  684.8357 ,  437.82858,
       1057.7839 ,  799.99695,  769.0972 ,  622.45795,  371.2307 ,
       1033.5789 ,  801.49945,  780.1343 ,  647.3353 ,  397.69742,
       1053.1469 ,  855.4624 ,  800.15063,  687.28613,  401.75623],
      dtype=float32)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/sample_submission.csv')
submit['중식계'] = lunch

In [ ]:
# 저녁
X = train_ln[['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bobd', 'soupd', 'maind',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]
y = train_ln['dinner_t']

feture_scaling(X, column = ['numbers', 'dayoff', 'work', 'outsidework', 'workfhome','bobd', 'soupd', 'maind',  'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화'])

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_vali, y_train, y_vali = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 

                                                    shuffle=True, 

                                                    random_state=1004)


In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[X.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mae", optimizer=optimizer)

    return model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
keras_reg

In [ ]:
# keras_reg.fit(X_train, y_train, epochs=1000,
#               validation_data=(X_vali, y_vali),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=1000,
                  validation_data=(X_vali, y_vali),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
21/21 [==============================] - 0s 6ms/step - loss: 410.8060 - val_loss: 386.6356
Epoch 686/1000
21/21 [==============================] - 0s 8ms/step - loss: 410.7199 - val_loss: 386.5513
Epoch 687/1000
21/21 [==============================] - 0s 8ms/step - loss: 410.6336 - val_loss: 386.4662
Epoch 688/1000
21/21 [==============================] - 0s 8ms/step - loss: 410.5465 - val_loss: 386.3819
Epoch 689/1000
21/21 [==============================] - 0s 6ms/step - loss: 410.4602 - val_loss: 386.2967
Epoch 690/1000
21/21 [==============================] - 0s 8ms/step - loss: 410.3732 - val_loss: 386.2121
Epoch 691/1000
21/21 [==============================] - 0s 8ms/step - loss: 410.2865 - val_loss: 386.1277
Epoch 692/1000
21/21 [==============================] - 0s 5ms/step - loss: 410.2002 - val_loss: 386.0437
Epoch 693/1000
21/21 [==============================] - 0s 6ms/step - loss: 410.1142 - val_loss: 385.9597
Epoch 694/1000
21/21 [===

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f0cf0244450>,
                   param_distributions={'learning_rate': [0.006232868841909281,
                                                          0.0072723434950311555,
                                                          0.014980025679402079,
                                                          0.000947233987395861,
                                                          0.0028574009317206828,
                                                          0.0008308835070579019,
                                                          0.028343747518194726,
                                                          0.02318675291623005,
                                                          0.0003597285254120405,
                                                          0.00773163956398385...
                                                          0.02320888266

In [ ]:
rnd_search_cv.best_params_

{'learning_rate': 0.008171782257177468, 'n_hidden': 1, 'n_neurons': 28}

In [ ]:
# 예측
dinner = rnd_search_cv.predict(test_dinner)

In [ ]:
dinner

array([336.20975, 405.3555 , 270.82422, 567.73737, 568.89087, 435.5234 ,
       523.34607, 425.26602, 604.28876, 565.8362 , 357.53696, 620.19476,
       629.5255 , 455.3533 , 567.465  , 427.20816, 639.0614 , 663.98724,
       470.6406 , 563.87964, 419.23468, 514.7158 , 368.7969 , 521.3014 ,
       305.82642, 577.13855, 563.095  , 355.57623, 440.7072 , 288.4557 ,
       520.4111 , 484.7813 , 310.77612, 400.19672, 292.37448, 500.83163,
       501.8304 , 344.97543, 433.91675, 308.7871 , 491.6701 , 491.3442 ,
       346.93542, 426.01044, 291.3574 , 489.0321 , 504.3117 , 336.09943,
       446.8015 , 287.57767], dtype=float32)

In [ ]:
submit['석식계'] = dinner

In [ ]:
submit.to_csv('Random_Grid_Search02.csv', index=False)

# Grid_search 


### 결과

- train : 182.5422686667
- test : 206.1309987143	




# 느낀점
- Random_Gridsearch 가 성능이 좋지 않을수도있다. 
    - 직접 파라미터를 조절하는게 훨씬 성능이 좋을수도 있다는 새각을 해봤다

- 생각보다 Random_Gridsearch 가 만능은 아니더라...
